# Experiments

In [120]:
import io

import ipywidgets as widgets
import pandas as pd

In [121]:
# *** constants ***

# All measurements expressed in meters unless noted
BREAKOFF_ALTITUDE = 1707.0
MAX_SPEED_ACCURACY = 3.0
PERFORMANCE_WINDOW_LENGTH = 2256.0
VALIDATION_WINDOW_LENGTH = 1006.0

In [122]:
uploader = widgets.FileUpload(description = 'Speed CSV', tooltip = 'FlySight speed file')

In [124]:
uploader

FileUpload(value=({'name': '11-02-19.CSV', 'type': 'text/csv', 'size': 1791611, 'content': <memory at 0xffff47…

In [125]:
data = pd.read_csv(io.BytesIO(uploader.value[0].content), skiprows= (1,1))

## Use meaningful column names

In [126]:
data['heightFt'] = data['hMSL'].apply(lambda h: 3.2808*h)

In [127]:
data['timeUnix'] = data['time'].apply(lambda t: pd.Timestamp(t).timestamp())

In [128]:
data = pd.DataFrame(data = {
    'timeUnix': data.timeUnix,
    'heightMSL': data.hMSL,
    'heightFt': data.heightFt,
    'vMetersPerSecond': data.velD,
    'vKMh': data.velD*3.6,
    'speedAccuracy': data.sAcc, })
data

,timeUnix,heightMSL,heightFt,vMetersPerSecond,vKMh,speedAccuracy
0,1.692468e+09,27.898,91.527758,-0.51,-1.836,3.90
1,1.692468e+09,29.006,95.162885,-0.47,-1.692,2.07
2,1.692468e+09,30.083,98.696306,-0.65,-2.340,1.60
3,1.692468e+09,29.927,98.184502,-0.72,-2.592,1.51
4,1.692468e+09,29.726,97.525061,-0.75,-2.700,1.42
...,...,...,...,...,...,...
15559,1.692470e+09,-14.061,-46.131329,1.50,5.400,1.45
15560,1.692470e+09,-14.309,-46.944967,1.66,5.976,1.45
15561,1.692470e+09,-14.307,-46.938406,1.62,5.832,1.41
15562,1.692470e+09,-14.266,-46.803893,1.57,5.652,1.39


## Discard non-actionable rows

Discard all rows before maximum altitude:

In [129]:
timeMaxAlt = data[data.heightMSL == data.heightMSL.max()].timeUnix.iloc[0]
data = data[data.timeUnix > timeMaxAlt]

Then discard all rows where height < 0; appears to be a bug in FlySight MSL altitude handling:

In [130]:
data = data[data.heightMSL > 0]

## Find the freefall data subset

In [131]:
def dataGroups(data):
    data_ = data.copy()
    data_['positive'] = (data_.vMetersPerSecond > 0)
    data_['group'] = (data_.positive != data_.positive.shift(1)).astype(int).cumsum()-1

    return data_

In [132]:
data = dataGroups(data)
groups = data.group.max()+1
print('Data groups = %d' % groups)

freeFallGroup = -1
dataPoints = -1
for group in range(groups):
    subset = data[data.group == group]
    if len(subset) > dataPoints:
        freeFallGroup = group
        dataPoints = len(subset)
freeFallGroup

data = data[data.group == freeFallGroup]
data = data.drop('group', axis = 1).drop('positive', axis = 1)

Data groups = 672


### Drop data before exit and below the breakoff altitude

Breakoff altitude is 1,707 meters - FAI ISC and USPA Competition rules.

In [133]:
data = data[data.vKMh >= 1.0]
data = data[data.heightMSL >= BREAKOFF_ALTITUDE]

### Identify performance, scoring, and validation window

The PERFORMANCE_WINDOW_LENGTH is 2,256 meters after exit
If the performance window is below the breakoff altitude, scoring ends at BREAKOFF_ALTITUDE

In [134]:
windowStart = data.iloc[0].heightMSL
windowEnd = windowStart-PERFORMANCE_WINDOW_LENGTH
if windowEnd < BREAKOFF_ALTITUDE:
    windowEnd = BREAKOFF_ALTITUDE

validationWindowStart = windowEnd+VALIDATION_WINDOW_LENGTH
data = data[data.heightMSL > windowEnd]

print('Window start = {0:,.2f}'.format(windowStart))
print('Validation window start = {0:,.2f}'.format(validationWindowStart))
print('Window end = {0:,.2f}'.format(windowEnd))

Window start = 4,190.09
Validation window start = 2,940.09
Window end = 1,934.09


---
## Jump validation
Every data sample within the validation window must satisfy the precision criterium of max speed accuracy < 3 m/s; 0.0 == most accurate.

In [135]:
speedAccuracy = data[data.heightMSL > validationWindowStart].speedAccuracy.max()

if speedAccuracy < MAX_SPEED_ACCURACY:
    color = '#0f0'
    result = 'valid'
else:
    color = '#f00'
    result = 'invalid'

validJumpStatus = '<h2><span style="color: %s">%s jump</span></h2>' % (color, result)
display(widgets.HTML(validJumpStatus))

HTML(value='<h2><span style="color: #0f0">valid jump</span></h2>')

---
## Jump analysis

In [138]:
data.iloc[0].timeUnix + 5

1692469206.5

In [140]:
data.query('timeUnix == 1692469207.5')

,timeUnix,heightMSL,heightFt,vMetersPerSecond,vKMh,speedAccuracy
10650,1.692469e+09,4165.627,13666.589062,15.2,54.72,0.52


In [184]:
table = None

In [185]:
columns = pd.Series([ '5', '10', '15', '20', '25', ])

In [186]:
for column in pd.Series([ '5', '10', '15', '20', '25', ]):
    timeOffset = data.iloc[0].timeUnix+float(column)
    tranche = data.query('timeUnix == %f' % timeOffset)
    if table is not None:
        table.append(tranche, ignore_index = True)
    else:
        table = tranche

AttributeError: 'DataFrame' object has no attribute 'append'

In [178]:
table

,timeUnix,heightMSL,heightFt,vMetersPerSecond,vKMh,speedAccuracy


In [179]:
table.describe()

,timeUnix,heightMSL,heightFt,vMetersPerSecond,vKMh,speedAccuracy
count,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN
